In [1]:
from IPython.core.events import EventManager
from IPython import get_ipython
import time

global_start_time = None

def start_timer(event):
    global global_start_time
    global_start_time = time.time()

def stop_timer(event):
    global global_start_time
    if global_start_time:
        duration = time.time() - global_start_time
        print(f"Cell execution time: {duration:.3f} seconds")
        global_start_time = None

ip = get_ipython()
ip.events.register('pre_run_cell', start_timer)
ip.events.register('post_run_cell', stop_timer)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import os, tifffile, cv2, zarr

from glob import glob

Cell execution time: 0.397 seconds


In [3]:
# proteins code mapping 
cprot_dict = {"cd3" : "CPROT00013",
              "cd68" : "CPROT00006",
              "cd163" : "CPROT00052",
              "cd19" : "CPROT00470",
              "vim" : "CPROT00168"}

Cell execution time: 0.000 seconds


In [4]:
# simple 8x4 fov set 
fovs = list(range(63, 71)) + list(range(42, 50)) + \
       list(range(22, 30)) + list(range(2, 10))

# 4x5 + 3x2 fov set 
# fovs = [249,250,251,252,
#         266,267,268,269,
#         283,284,285,286,
#         299,300,301,302,
#         312,313,314,315,
#         325,326,327,335,336,337]

# set of fovs not well aligned and disconnected 
# fovs = list(range(358,382))

Cell execution time: 0.000 seconds


In [5]:
# channels and path to channels  
channels   = ["cd3", "cd19", "cd163", "cd68", "vim"]   
CH_DIR_OUT = {c: f"/Users/jonathanperrie/Documents/UCLA/projects/cosmx/protein/data/ptb211_zenodo/{c}" for c in channels}            

# tiffs are 4256 x 4256
FOV_SIZE   = 4256  
# downsample scale 
SCALE      = 0.25                         

# file name pattern 
PATTERN = "20240120_024046_S3_C001_F{fov:03d}"

Cell execution time: 0.000 seconds


In [6]:
# fov coords 
# FOV,x_global_px,y_global_px,x_global_mm,y_global_mm
df   = pd.read_csv("data/PTB211protein_fov_positions_file.csv")    
df = df.loc[[True if x in fovs else False for x in df.FOV.values]] 
coords = {int(r.FOV): (int(r.x_global_px), int(r.y_global_px))
          for r in df.itertuples()}

fovs = list(coords.keys())             

Cell execution time: 0.003 seconds


In [7]:
# calculate the shape of the zarr matrix
min_x  = min(x for x, _ in coords.values())
min_y  = min(y for _, y in coords.values())
max_x  = max(x for x, _ in coords.values()) + FOV_SIZE
max_y  = max(y for _, y in coords.values()) + FOV_SIZE

mosaic_w = int(np.ceil((max_x - min_x) * SCALE))
mosaic_h = int(np.ceil((max_y - min_y) * SCALE))

OUT_SHAPE  = (len(channels),      
              mosaic_h,
              mosaic_w)   

Cell execution time: 0.000 seconds


In [8]:
# create empty zarr matrix 
store = zarr.DirectoryStore("mosaic.zarr")
root  = zarr.group(store=store, overwrite=True)
mosaic = root.create_dataset("mosaic",
                             shape=OUT_SHAPE,
                             chunks=(len(channels), 1024, 1024),
                             dtype="uint16")

Cell execution time: 0.005 seconds


In [9]:
def get_coords(fov):
    """
    For a given fov, return coords (flip y)
    """
    SIDE_PX = int(FOV_SIZE * SCALE)
    x_native, y_native = coords[fov]
    
    # offset from the global origin, then downsample
    x_off = int(round((x_native - min_x) * SCALE))
    y_off = int(round((y_native - min_y) * SCALE))
    return (
        slice(mosaic_h - y_off - SIDE_PX, mosaic_h - y_off),
        slice(x_off, x_off + SIDE_PX),
    )

Cell execution time: 0.000 seconds


In [10]:
def read_channel(fov, chan):
    """Load tiff and return downsample"""
    # file path of tiff 
    fp = os.path.join(CH_DIR_OUT[chan], PATTERN.format(fov=fov) +"_" + cprot_dict[chan] + ".TIF")
    
    if not os.path.exists(fp):
        raise FileNotFoundError(fp)
        
    img = tifffile.imread(fp)         
    side = int(FOV_SIZE * SCALE)
    
    return cv2.resize(img, (side, side),
                      interpolation=cv2.INTER_NEAREST)

Cell execution time: 0.000 seconds


In [11]:
for fov in fovs:
    ysl, xsl = get_coords(fov)
    for cidx, chan in enumerate(channels):
        mosaic[cidx, ysl, xsl] = read_channel(fov, chan)
    # sanity check 
    # print(f"[Placed] FOV {fov} at Y{ysl.start}:{ysl.stop}, X{xsl.start}:{xsl.stop}")


Cell execution time: 14.697 seconds


In [12]:
selected_mat = np.array(mosaic[:,:,:])

Cell execution time: 0.126 seconds


In [13]:
cd3_floor,    cd3_ceil    = 20, 800
cd19_floor,   cd19_ceil   = 20, 1500
cd163_floor,  cd163_ceil  = 20, 2000
cd68_floor,   cd68_ceil   = 20, 1500
vim_floor,    vim_ceil    = 100, 5000
 
floor_values = [cd3_floor, cd19_floor, cd163_floor, cd68_floor, vim_floor]
ceil_values  = [cd3_ceil,  cd19_ceil,  cd163_ceil,  cd68_ceil,  vim_ceil ]

# map proteins to rgb 
# cd3 to yellow 
# cd19 to green 
# cd163 to red 
# cd68 to blue
# vimentin to grey
channel_colors = [
    (1.0, 1.0, 0.0),  
    (0.0, 1.0, 0.0),  
    (1.0, 0.0, 0.0),  
    (0.0, 0.0, 1.0),  
    (0.5, 0.5, 0.5)   
]

Cell execution time: 0.000 seconds


In [14]:
# rescale to floor/ceil and then normalize by diff to 0-1 space
scaled_channels = []
for i in range(selected_mat.shape[0]):
    chan = selected_mat[i, :, :].astype(np.float32)

    # clip
    chan_clipped = np.clip(chan, floor_values[i], ceil_values[i])

    # rescale 
    chan_scaled = (chan_clipped - floor_values[i]) / (ceil_values[i] - floor_values[i])
    
    scaled_channels.append(chan_scaled)

scaled_channels = np.array(scaled_channels)  

Cell execution time: 0.235 seconds


In [15]:
height, width = scaled_channels.shape[1], scaled_channels.shape[2]
rgb_image = np.zeros((height, width, 3), dtype=np.float32)

# build composite 
for i in range(len(channels)):
    ccolor = channel_colors[i]    # rgb vec e.g. (1,1,0)
    cscaled = scaled_channels[i]  # normalized channel (width x height)

    # add composites 
    rgb_image[..., 0] += cscaled * ccolor[0]
    rgb_image[..., 1] += cscaled * ccolor[1]
    rgb_image[..., 2] += cscaled * ccolor[2]

# avoid saturation above 1 
rgb_image = np.clip(rgb_image, 0.0, 1.0)

Cell execution time: 0.402 seconds


In [16]:
rgb_image_8bit = (rgb_image * 255).astype(np.uint8)
tifffile.imwrite("plots/cosmx_sample_plots/composite_image1.tif", np.flip(rgb_image_8bit, axis=0))

Cell execution time: 0.073 seconds
